<a href="https://colab.research.google.com/github/saha23s/MHC-Data-Sync/blob/main/CleanVisionToForceModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Oct  8 19:54:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

We are working on creating a Machine Learning model to learn the pattern in the vision and force data so that the force values are predicted from the images provided.

We have a multi variate dataset.

Vision Data:
We have four images coming from four different cameras at a same timestep.
Force Data:
We have 12 different datapoint at a same timestep

Our goal:
Since our dataset is temporal, we want the model to learn accross 4 timesteps.

Approach:
A combination of Convolutional Neural Networks (CNN) and Long Short-Term Memory (LSTM) networks

High Level Overview:
1. A CNN for feature extraction from the images.
2. An LSTM layer to capture the temporal dependencies in the sequence of extracted features over the four timesteps.
3. Dense layers to combine the output from the LSTM with the force values and to make the final prediction.

Step 1: Structure The Data

Organize the data in a manner that groups the four images at each timestep together.
Associate the corresponding force data with each timestep.

1.1 Load force data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Lambda, Bidirectional
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')

# Let's get the final xsv files
final = [
 'flarp_folding_1_final.csv',
#  'flarp_folding_2_final.csv',
#  'flarp_folding_3_final.csv',
#  'flarp_folding_4_final.csv',
#  'flarp_folding_5_final.csv',
#  'flarp_pull_1_final.csv',
#  'flarp_pull_5_final.csv',
#  'flarp_push_1_final.csv',
#  'flarp_push_3_final.csv',
#  'flarp_push_4_final.csv',
#  'flarp_push_5_final.csv',
#  'flarp_sponge_push_1_final.csv',
#  'flarp_sponge_push_2_final.csv',
#  'flarp_sponge_push_3_final.csv',
#  'flarp_sponge_push_4_final.csv',
#  'flarp_sponge_push_5_final.csv'
]
good_datasets = [pd.read_csv('/content/drive/MyDrive/MelodyResearch/final_csv/final_csv/' + name) for name in final]

force_df = good_datasets[0]

# the below file is for testing file
final_2 = ['flarp_folding_2_final.csv']

test = [pd.read_csv('/content/drive/MyDrive/MelodyResearch/final_csv/final_csv/' + name) for name in final_2]

test_df = test[0]

# Specify the force columns to extract
force_columns = [
    'field.base.tool_external_wrench_force_x',
    'field.base.tool_external_wrench_force_y',
    'field.base.tool_external_wrench_force_z',
    'field.base.tool_external_wrench_torque_x',
    'field.base.tool_external_wrench_torque_y',
    'field.base.tool_external_wrench_torque_z'
    # for now only predicting the first 6
    # 'field.actuators0.torque',
    # 'field.actuators1.torque',
    # 'field.actuators2.torque',
    # 'field.actuators3.torque',
    # 'field.actuators4.torque',
    # 'field.actuators5.torque'
]

# Extract and structure the force values according to the %time column
num_timesteps = 4  # Number of timesteps in each sequence

# Calculate number of sequences (samples) in the dataset
num_samples = len(force_df) // num_timesteps
num_samples_2 = len(test_df)// num_timesteps

# Initialize a 3D numpy array to hold the force values
force_values = np.zeros((num_samples, num_timesteps, len(force_columns)))
test_values = np.zeros((num_samples_2, num_timesteps, len(force_columns)))

for i in range(num_samples):
    start_idx = i * num_timesteps
    end_idx = start_idx + num_timesteps
    force_values[i] = force_df.loc[start_idx:end_idx - 1, force_columns].to_numpy()

for i in range(num_samples_2):
    start_idx = i * num_timesteps
    end_idx = start_idx + num_timesteps
    test_values[i] = test_df.loc[start_idx:end_idx - 1, force_columns].to_numpy()

print("Force Values Shape:", force_values.shape)
print("Test Values Shape:", test_values.shape)

Mounted at /content/drive
Force Values Shape: (182, 4, 6)
Test Values Shape: (176, 4, 6)


In [ ]:
# Print the force values for the first sequence (first 4 timesteps)
print("Force values for the first sequence:")
print(force_values[0])
print()

Force values for the first sequence:
[[ 0.09746455 -0.74159235  0.38882917 -0.92945951 -0.3058013   0.03083278]
 [ 0.15739678 -0.62655967  0.35027048 -1.00680876 -0.30121917  0.02171012]
 [ 0.09304701 -0.66954929  0.42921522 -0.94887626 -0.31210107  0.03340989]
 [ 0.07041009 -0.75054866  0.44438002 -0.91801381 -0.33097449  0.02956343]]


In [ ]:
import numpy as np
# Initialize a list to hold the loaded images
num_samples = force_values.shape[0]  # Number of sequences (samples) in the dataset
num_timesteps = 4  # Number of timesteps in each sequence
num_images = 4  # Number of images at each timestep
height, width, channels = 224, 224, 3  # Dimensions of the images

# Initialize a 5D numpy array to hold the image data
images = np.zeros((num_samples, num_timesteps, num_images, height, width, channels))


In [ ]:
import cv2
import os

# Base path to your image folders
base_path = '/content/drive/MyDrive/MelodyResearch/WholeData/experimentData/'
csv_folder_path = '/content/drive/MyDrive/MelodyResearch/final_csv/final_csv/'

# Outer and inner folder names
outer_folders = ['flarp_folding_1']
inner_folders = ['kinova_color_images', 'external_depth_images', 'kinova_depth_images', 'external_color_images']

# Iterate over the sequences (samples) and timesteps to load and structure the images
for i in range(num_samples):
    for j in range(num_timesteps):
        # Calculate the index in the original dataframe
        idx = i * num_timesteps + j

        # Extract the corresponding image frame from the dataframe
        image_frame = force_df.loc[idx, 'image_frames']

        # Load and structure the images for the current timestep
        for k, inner_folder in enumerate(inner_folders):
            for outer_folder in outer_folders:
                image_folder_path = os.path.join(base_path, outer_folder, inner_folder)
                image_path = os.path.join(image_folder_path, image_frame)
                print(image_path)

                # Check if the image exists
                if os.path.exists(image_path):
                    # Load and preprocess the image
                    img = cv2.imread(image_path)
                    img = cv2.resize(img, (height, width))  # Resize the image to the required dimensions
                    img = img / 255.0  # Normalize pixel values to [0, 1]

                    # Assign the loaded image to the corresponding position in the images array
                    images[i, j, k] = img
                else:
                    print(f"Image not found: {image_path}")

print("Images Array Shape:", images.shape)
np.save('/content/drive/MyDrive/MelodyResearch/processed_images.npy', images)


/content/drive/MyDrive/MelodyResearch/WholeData/experimentData/flarp_folding_1/kinova_color_images/myframe000000.png
/content/drive/MyDrive/MelodyResearch/WholeData/experimentData/flarp_folding_1/external_depth_images/myframe000000.png
/content/drive/MyDrive/MelodyResearch/WholeData/experimentData/flarp_folding_1/kinova_depth_images/myframe000000.png
/content/drive/MyDrive/MelodyResearch/WholeData/experimentData/flarp_folding_1/external_color_images/myframe000000.png
/content/drive/MyDrive/MelodyResearch/WholeData/experimentData/flarp_folding_1/kinova_color_images/myframe000001.png
/content/drive/MyDrive/MelodyResearch/WholeData/experimentData/flarp_folding_1/external_depth_images/myframe000001.png
/content/drive/MyDrive/MelodyResearch/WholeData/experimentData/flarp_folding_1/kinova_depth_images/myframe000001.png
/content/drive/MyDrive/MelodyResearch/WholeData/experimentData/flarp_folding_1/external_color_images/myframe000001.png
/content/drive/MyDrive/MelodyResearch/WholeData/experime

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Reshape, Flatten

In [ ]:
def create_cnn(input_shape):
    model = Sequential()
    model.add(Conv2D(16, (3, 3), activation='relu', input_shape=input_shape))  # Reduced number of filters
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu'))  # Additional Conv layer
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))  # Additional Dense layer to reduce dimensionality
    return model

def create_model(input_shape, num_force_features):
    image_input = Input(shape=input_shape)

    cnn = create_cnn(input_shape[2:])
    encoded_frames = TimeDistributed(TimeDistributed(cnn))(image_input)

    # Concatenate the features from the four images at each timestep
    encoded_frames = Flatten()(encoded_frames)
    encoded_frames = Reshape((input_shape[0], -1))(encoded_frames)

    lstm_out = LSTM(50, return_sequences=True, recurrent_activation='sigmoid', unroll=True)(encoded_frames)
    lstm_out = LSTM(50, recurrent_activation='sigmoid', unroll=True)(lstm_out)

    output = Dense(num_force_features)(lstm_out)

    model = Model(inputs=image_input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    return model




In [ ]:
# Get the shapes of the images and force values arrays
num_timesteps, num_images, height, width, channels = images.shape[1:]
num_force_features = force_values.shape[-1]

# Create the model
model = create_model((num_timesteps, num_images, height, width, channels), num_force_features)

from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
images_train, images_val, force_values_train, force_values_val = train_test_split(
    images, force_values[:, -1, :], test_size=0.2, random_state=42
)

# Train the model using the training data
model.fit(images_train, force_values_train, epochs=10, batch_size=32)

# Evaluate the model on the validation data
val_loss = model.evaluate(images_val, force_values_val)
print(f'Validation Loss: {val_loss}')

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt

# Predict the force values on the validation set
predictions = model.predict(images_val)

# Calculate additional metrics
mae = mean_absolute_error(force_values_val, predictions)
r2 = r2_score(force_values_val, predictions)

print(f'Mean Absolute Error: {mae}')
print(f'R^2 Score: {r2}')

# Visual Inspection: Plotting true vs predicted values
plt.scatter(force_values_val, predictions)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.title('True vs Predicted Force Values')
plt.show()

# Plot residuals
residuals = force_values_val - predictions
plt.hist(residuals, bins=30)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Distribution of Residuals')
plt.show()